In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import warnings
warnings.filterwarnings('ignore')

In [2]:
#credit for this code: https://www.hackerearth.com/fr/practice/notes/praveen97uma/crawling-a-website-that-loads-content-using-javascript-with-selenium-webdriver-in-python/
#this code creates a function that gets the browser to scroll down
def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

In [3]:
chrome_path = "C:/Users/Nadir/Desktop/B/Projects/Fragrance_project/chromedriver"
driver = webdriver.Chrome(executable_path = chrome_path)


#this list represents different skin care categories to collect
categories = ['fragrance']
#creating an empty data frame that all the URLs and categories will go in to
final_df = pd.DataFrame(columns = ['category', 'URL'])


for category in categories: #looping through the different categories

    page_num = 1 #the first page

    while True:
        
        print("Progress: {}".format("" + str(len(final_df))) )
    
        #when running the following .get request in order for everything to work properly
        #the window needs to be popped open on the screen
        url = 'https://www.sephora.com/shop/'+ category + '?pageSize=300&currentPage=' + str(page_num)
        driver.get(url)
        time.sleep(15)
        try: #check to see if the page is empty   
            if driver.find_element_by_class_name('css-2td14d.e65zztl0').is_displayed():
                break
            
        except:
    
            #check to see if there is a pop up windew
            try:
                #exit the pop up window
                #btn = driver.find_element_by_xpath('//*[@id="modalDialog"]/button').click()
                btn = driver.find_element_by_class_name('css-1kna575').click()
                time.sleep(10)
            except:
                pass
    
            #as scrolling check if there is any more room to scroll
            old_len = 0
            while True:
                browser = scrollDown(driver, 20) #scroll down the page
                time.sleep(10) #give it time to load
                slug = driver.find_elements_by_class_name('css-klx76') #look for the urls of products
                new_len = len(slug)
                if old_len == new_len: #if the old length and new length are equal, the bottom of page was reached
                    break
                else:
                    old_len = new_len

            #from the list of URLs in slug pull all the href and make a dictionary with it and the category name        
            slugURL = []
            for a in slug:
                subURL = {}
                subURL['category'] = category 
                subURL['URL'] = a.get_attribute('href')
                slugURL.append(subURL)
    
            #append our data frame with categories and URLs
            df = pd.DataFrame(slugURL)
            #print(df.head())
            #adding to go to next page
            page_num += 1
            #concatenating to get all in same df
            final_df = pd.concat([final_df, df], axis = 0, ignore_index = True) 
    
#close the chrome
driver.close()

Progress: 0
Progress: 100
Progress: 200
Progress: 300
Progress: 400
Progress: 500
Progress: 600
Progress: 700
Progress: 800
Progress: 900
Progress: 1000
Progress: 1100
Progress: 1200
Progress: 1300
Progress: 1400
Progress: 1438


In [4]:
#checking the structure of my final dataframe
final_df

,category,URL
0,fragrance,https://www.sephora.com/product/sol-de-janeiro...
1,fragrance,https://www.sephora.com/product/brazilian-crus...
2,fragrance,https://www.sephora.com/product/chance-eau-ten...
3,fragrance,https://www.sephora.com/product/yves-saint-lau...
4,fragrance,https://www.sephora.com/product/sol-de-janeiro...
...,...,...
1433,fragrance,https://www.sephora.com/product/velvet-pear-vo...
1434,fragrance,https://www.sephora.com/product/mandarino-di-a...
1435,fragrance,https://www.sephora.com/product/ralph-lauren-r...
1436,fragrance,https://www.sephora.com/product/19-69-purple-h...


In [8]:
#saving the final dataframe of URLs in to a csv folder
final_df.to_csv('C:/Users/Nadir/Desktop/B/Projects/Fragrance_project/data/product_urls.csv', index = False)